## 8: Building a Streamlit Frontend (OPTIONAL)

This notebook builds a customer-facing Streamlit web application with secure authentication, real-time chat, and session management. You'll launch the app and test the complete customer experience.

![Streamlit](images/Streamlit.png)

---

**Prerequisites:** Completed 4, agent deployed and running

### Step 1: Install Frontend Dependencies

In [ ]:
# Install frontend-specific dependencies
%pip install -r utils/streamlit_frontend/requirements.txt -q
print("✅ Frontend dependencies installed successfully!")

### Step 2: Understanding the Frontend Architecture

The application consists of:
- **main.py**: Streamlit UI and authentication
- **chat.py**: Chat management and Runtime integration
- **chat_utils.py**: Message formatting utilities
- **sagemaker_helper.py**: URL generation helper

### Step 2.5: Configure SSM Parameters

The Streamlit app needs the agent ARN stored in SSM Parameter Store. Let's set it up:

In [ ]:
import json
import boto3
import os

# Get the notebook directory (in case we changed directories)
notebook_dir = os.path.dirname(os.path.abspath('__file__')) if '__file__' in dir() else os.getcwd()
# Navigate to parent directory if we're in streamlit_frontend
if notebook_dir.endswith('streamlit_frontend'):
    notebook_dir = os.path.dirname(os.path.dirname(notebook_dir))
elif notebook_dir.endswith('utils'):
    notebook_dir = os.path.dirname(notebook_dir)

config_path = os.path.join(notebook_dir, 'runtime_config.json')

# Load runtime configuration from 5
with open(config_path, 'r') as f:
    runtime_config = json.load(f)

agent_arn = runtime_config['agent_arn']

# Store agent ARN in SSM Parameter Store
ssm = boto3.client('ssm', region_name='us-west-2')

try:
    ssm.put_parameter(
        Name='/app/returnsrefunds/agentcore/runtime_arn',
        Value=agent_arn,
        Type='String',
        Overwrite=True,
        Description='Agent ARN for Returns and Refunds Streamlit app'
    )
    print(f"✅ Stored agent ARN in SSM Parameter Store")
    print(f"   Parameter: /app/returnsrefunds/agentcore/runtime_arn")
    print(f"   Value: {agent_arn}")
except Exception as e:
    print(f"❌ Error storing parameter: {e}")

### Step 3: Launch the Application

Start the Streamlit server on port 8501. The app will connect to your deployed AgentCore Runtime.

**Note:** The app runs continuously until stopped (Ctrl+C). Cognito tokens are valid for 2 hours.

In [ ]:
import subprocess
import time
import os

print("🚀 Starting Returns and Refunds Agent Streamlit Application...")
print("=" * 60)

# Check if streamlit is already running
result = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True)

if result.returncode == 0:
    print("\n✅ Streamlit is already running!")
    print("   Access it at: http://localhost:8501")
else:
    print("\n🔄 Starting Streamlit app...")
    # Start streamlit in background
    os.chdir('utils/streamlit_frontend')
    subprocess.Popen(['streamlit', 'run', 'main.py', '--server.port', '8501', 
                      '--server.address', '0.0.0.0', '--server.headless', 'true'],
                     stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)
    print("   ✅ Streamlit started!")
    print("   Access it at: http://localhost:8501")

print("\n" + "=" * 60)

### Get the Application URL and Credentials

In [ ]:
# Display the Streamlit URL
print('\n🚀 Returns and Refunds Agent Streamlit Application URL:')
print('   http://localhost:8501\n')
print("Please use the following to login and test the Streamlit Application")
print("Username:       testuser")
print("Password:       MyPassword123!")

### Step 4: Testing Your Application

Once running, test the customer support experience:

1. Access the URL and sign in with test credentials
2. Verify the welcome message appears
3. Test return policy questions
4. Verify memory and context are maintained

### Sample Prompt to validate
- Return Policy Questions: I bought a Kindle Book three days ago by accident in India. I want to get a refund, what date is the ETA if I request it now?"

![Login Screen](images/streamlit_login.png)
![Agent Question](images/agent_question.png)

### Step 5: Customization Options

Customize by modifying:
- **main.py**: Colors, themes, branding, layout
- **chat.py**: Features, behavior, tool integration
- **chat_utils.py**: Messages, error handling, formatting

### Stopping the Application

To stop the Streamlit application:

In [ ]:
import subprocess

# Stop streamlit process
result = subprocess.run(['pkill', '-f', 'streamlit'], capture_output=True)

if result.returncode == 0:
    print("✅ Streamlit application stopped")
else:
    print("ℹ️ No Streamlit process found running")

### Summary

You've built a complete customer-facing application with secure authentication, real-time chat, and session management. The frontend seamlessly integrates with your deployed AgentCore Runtime.

### Next Steps

- Customize styling and branding
- Add multi-language support
- Integrate with CRM or ticketing systems
- Optimize for mobile devices